In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "DragonBall123"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

def load_df(mongo_query: dict):
    """Helper: run a read() and return a clean dataframe."""
    docs = db.read(mongo_query if mongo_query else {})
    df_local = pd.DataFrame.from_records(docs)
    if not df_local.empty and "_id" in df_local.columns:
        df_local.drop(columns=["_id"], inplace=True)
    return df_local

# Start with ALL records
df = load_df({})

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# ---- Branding: Grazioso logo + link + unique identifier ----
# Put your logo file (PNG) next to this notebook and set its filename here:
logo_filename = "grazioso_logo.png" 
logo_src = None
if os.path.exists(logo_filename):
    with open(logo_filename, "rb") as f:
        encoded = base64.b64encode(f.read()).decode()
        logo_src = f"data:image/png;base64,{encoded}"

# Rescue filter radio options (spec)
FILTER_OPTIONS = [
    {"label": "Water Rescue", "value": "water"},
    {"label": "Mountain Rescue", "value": "mountain"},
    {"label": "Disaster Rescue", "value": "disaster"},
    {"label": "Reset (All)", "value": "reset"},
]

app.layout = html.Div([
    html.Div([
        html.H1("SNHU CS-340 Dashboard — Carlos Trujillo", style={"margin": 0}),
        html.A(
            html.Img(src=logo_src, style={"height": "48px"}) if logo_src else html.Span("Grazioso Salvare"),
            href="https://www.snhu.edu",
            target="_blank",
            style={"display": "inline-block", "marginLeft": "12px"}
        ),
    ], style={"display": "flex", "alignItems": "center", "gap": "12px"}),

    html.Hr(),

    # -------- Interactive filter controls --------
    html.Div([
        html.H3("Filter Options"),
        dcc.RadioItems(
            id="filter-type",
            options=FILTER_OPTIONS,
            value="reset",
            inline=True
        )
    ], style={"border": "2px solid #673ab7", "padding": "10px", "borderRadius": "8px"}),

    html.Hr(),

    # -------- Interactive Data Table --------
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={"overflowX": "auto"},
        style_cell={"minWidth": 80, "maxWidth": 300, "whiteSpace": "normal"},
        style_header={"fontWeight": "bold"},
    ),

    html.Br(),
    html.Hr(),

    # -------- Charts row --------
    html.Div(className='row', style={'display': 'flex', 'gap': '16px'}, children=[
        html.Div(id='graph-id', className='col s12 m6', style={"flex": 1}),
        html.Div(id='map-id', className='col s12 m6', style={"flex": 1}),
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# ----- Query builders for filters (per spec table) -----
def query_for_filter(filter_value: str) -> dict:
    """
    Build a MongoDB query for each rescue type using:
    - Preferred breeds
    - Preferred sex
    - Training age window in weeks (age_upon_outcome_in_weeks)
    """
    if filter_value == "water":
        breeds = [
            "Labrador Retriever Mix",
            "Chesapeake Bay Retriever",
            "Newfoundland"
        ]
        sex = "Intact Female"
        min_w, max_w = 26, 156
    elif filter_value == "mountain":
        breeds = [
            "German Shepherd",
            "Alaskan Malamute",
            "Old English Sheepdog",
            "Siberian Husky",
            "Rottweiler"
        ]
        sex = "Intact Male"
        min_w, max_w = 26, 156
    elif filter_value == "disaster":
        breeds = [
            "Doberman Pinscher",
            "German Shepherd",
            "Golden Retriever",
            "Bloodhound",
            "Rottweiler"
        ]
        sex = "Intact Male"
        min_w, max_w = 20, 300
    else:
        # reset -> no filter
        return {}

    return {
        "breed": {"$in": breeds},
        "sex_upon_outcome": sex,
        "age_upon_outcome_in_weeks": {"$gte": min_w, "$lte": max_w},
        "animal_type": "Dog"
    }

# Update table data when filter changes
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    query = query_for_filter(filter_type)
    dff = load_df(query)
    return dff.to_dict('records')

# Pie chart of breeds from current table view
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame(viewData) if viewData is not None else df.copy()
    if dff.empty or "breed" not in dff.columns:
        return [html.Div("No data to chart.")]
    # Build pie by breed
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return [dcc.Graph(figure=fig)]

# Highlight selected column(s) (kept from starter)
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in (selected_columns or [])]

# Update map based on selected row in the current table view
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData:
        return [html.Div("No data to map.")]
    dff = pd.DataFrame.from_dict(viewData)
    # Ensure we have coordinate columns present (dataset adds location_lat/long)
    # If your columns are differently named, adjust the indices/keys below.
    # Fallback to Austin center if selection invalid.
    if not index:
        row = 0
    else:
        row = index[0]

    # Defensive defaults
    center_lat, center_lon = 30.75, -97.48
    try:
        # Prefer explicit columns if available; otherwise match the starter’s iloc positions
        if {"location_lat", "location_long"}.issubset(dff.columns):
            lat = float(dff.iloc[row]["location_lat"])
            lon = float(dff.iloc[row]["location_long"])
            name = dff.iloc[row].get("name", "Unknown")
            breed = dff.iloc[row].get("breed", "Unknown")
        else:
            # Matches original starter’s index assumptions (13:lat, 14:long, 4:breed, 9:name)
            lat = float(dff.iloc[row, 13])
            lon = float(dff.iloc[row, 14])
            breed = dff.iloc[row, 4]
            name = dff.iloc[row, 9]
        center_lat, center_lon = lat, lon
    except Exception:
        name, breed = "Unknown", "Unknown"

    return [
        dl.Map(style={'width': '100%', 'height': '500px'},
               center=[center_lat, center_lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[center_lat, center_lon], children=[
                dl.Tooltip(str(breed)),
                dl.Popup([
                    html.H4("Animal Name"),
                    html.P(str(name))
                ])
            ])
        ])
    ]

# For Jupyter: show inline
app.run_server(mode="inline", debug=False)


 * Running on http://127.0.0.1:12508/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Aug/2025 21:09:36] "GET /_alive_5cfde280-3241-43ce-9c24-fbab79acbf7c HTTP/1.1" 200 -


127.0.0.1 - - [14/Aug/2025 21:09:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2025 21:09:36] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2025 21:09:36] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2025 21:09:36] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Aug/2025 21:09:36] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Aug/2025 21:09:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2025 21:09:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2025 21:09:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2025 21:09:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2025 21:09:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2025 21:09:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2025 21:09:37] "GET /_dash-component-sui